# q3dfit notebook: NIRSPEC, J1652, convolution

<h3><font color='teal'>Installation of the environment and the package are described <a href="https://q3dfit.readthedocs.io/">here</a>. </font></h3>

This Jupyter notebook allows you to run Q3Dfit, a PSF decomposition and spectral analysis package tailored for JWST NIRSpec and MIRI IFU observations. 

Q3Dfit is developed as a science-enabling data product by the Early Release Science Team #1335 Q3D. You can find more information about this ERS program **Q3D** [here](https://wwwstaff.ari.uni-heidelberg.de/dwylezalek/q3d.html) and [here](https://www.stsci.edu/jwst/science-execution/approved-programs/dd-ers/program-1335).

The software is based on the existing package IFSFIT developed by Dave Rupke (see [ADS link](https://ui.adsabs.harvard.edu/abs/2017ApJ...850...40R/abstract)).

The following notebook will guide you through the initialization procedure and will then perform the analysis. 

## Table of Contents

* [1. Initialization](#chapter1)
    * [1.0. Setting up the directory tree](#chapter1_0)
    * [1.1. Setting up the data and models](#chapter1_1)
    * [1.2. Setting up the fitting parameters](#chapter1_2)
        * [1.2.1. Emission line parameters](#chapter1_2_1)
        * [1.2.2. Continuum parameters](#chapter1_2_2)
    * [1.3. Setting up the plotting parameters](#chapter1_3)
* [2. Run fitting and plotting routines](#chapter2)

## 1. Initialization <a class="anchor" id="chapter1"></a>

In [ ]:
import os.path
import numpy as np
%matplotlib inline
# unless you are running this python notebook from the directory where q3dfit is installed, you need to point the
# path to the package in order to import it. 
import sys
sys.path.append("")
sys.path.append("../")

In [ ]:
# Convenience function
# Ref: https://github.com/spacetelescope/jwebbinar_prep/blob/main/imaging_mode/imaging_mode_stage_1.ipynb
from astropy.utils.data import download_file
def download_files(files, output_directory, force=False):
    """Given a tuple or list of tuples containing (URL, filename),
    download the given files into the current working directory.
    Downloading is done via astropy's download_file. A symbolic link
    is created in the specified output dirctory that points to the
    downloaded file.
    
    Parameters
    ----------
    files : tuple or list of tuples
        Each 2-tuple should contain (URL, filename), where
        URL is the URL from which to download the file, and
        filename will be the name of the symlink pointing to
        the downloaded file.
        
    output_directory : str
        Name of the directory in which to create the symbolic
        links to the downloaded files
        
    force : bool
        If True, the file will be downloaded regarless of whether
        it is already present or not.
                
    Returns
    -------
    filenames : list
        List of filenames corresponding to the symbolic links
        of the downloaded files
    """
    # In the case of a single input tuple, make it a
    # 1 element list, for consistency.
    filenames = []
    if isinstance(files, tuple):
        files = [files]
        
    for file in files:
        filenames.append(file[1])
        if force:
            print('Downloading {}...'.format(file[1]))
            demo_file = download_file(file[0], cache='update')
            # Make a symbolic link using a local name for convenience
            if not os.path.islink(os.path.join(output_directory, file[1])):
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
        else:
            if not os.path.isfile(os.path.join(output_directory, file[1])):
                print('Downloading {}...'.format(file[1]))
                demo_file = download_file(file[0], cache=True)
                # Make a symbolic link using a local name for convenience
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
            else:
                print('{} already exists, skipping download...'.format(file[1]))
                continue
    return filenames    

### 1.0. Setting up the directory tree <a class="anchor" id="chapter1_0"></a>

Define the directories in which the data cube(s) that you want to analyse are stored and the output directories. We recommend creating a working directory that you name after your target, in which all outputs from q3dfit will be saved. Then download test data.

In [ ]:
# Base directory (book-keeping)
volume = 'nirspec-j1652-q3d_conv_fullcube3comp/'
# prefix label for output files
gal = 'nirspec-j1652-q3d_conv'
# Input directory
indir = volume
if not os.path.exists(indir):
    os.makedirs(indir)
# Output directory
outdir = volume
if not os.path.exists(outdir):
    os.makedirs(outdir)
# Initialization file (q3di.npy) directory
initdir = volume
# Output logfile
logfile = outdir+gal+'-fitlog.txt'

Download data from public Box folder:

In [ ]:
# make tuples of urls and download filenames
# infile = data cube
infile_tup=('https://rhodes.box.com/s/6c6pvi8gbbhaxkkvolgpp7ryuxp7z1is','nirspec-j1652-conv.fits')
# download files; by default don't force overwrite and take first element of output
infile = download_files(infile_tup, outdir, force=False)[0]
# add subdirectory to filenames
infile = initdir+infile
print(infile)

### 1.1. Setting up the data and models <a class="anchor" id="chapter1_1"></a>

Some general information about your cube. `argsreadcube` is a dictionary of attributes sent to the `CUBE` class.
- For non-JWST data, set `wmapext` to `None`. The WMAP extension is a [3-D weight image](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/science_products.html) giving the relative weights of the output spaxels.
- Microns are the wavelength unit used internally, but `q3dfit` can accept other wavelength input units or output other units.
- The code does calculations in flambda space, but assumes input units of MJy/sr, the JWST default. Other input wavelength units can be specified.

In [ ]:
from q3dfit.readcube import Cube
argscube = {'wmapext': None, 'pixarea_sqas': 0.0025, 'fluxnorm': 1e-17, 'datext': 0, 'varext': 1, 'dqext': 2, 'fluxunit_in' : 'erg/s/cm2/A'}
if not os.path.isfile(infile): 
    print('Data cube not found.')
else:
    cube = Cube(infile, **argscube)
    ncols = cube.ncols # number of columns
    nrows = cube.nrows # number of rows
    cube.about()

Name and systemic redshift of the galaxy. `zsys` is presently not used within the code, but would be an input for, e.g., calculating velocity maps in routines under development (e.g., `makemaps.py`). It is used in this notebook in initializing the arrays of initial guesses below.

In [ ]:
name = 'J1652'
zsys_gas = 2.9489

Wavelength range over which to fit data and the range of wavelength to ignore:

In [ ]:
fitrange = [1.66,3.17] #in um
cutrange = np.array([2.375,2.47])


Generating the quasar template used for PSF subtraction:


In [ ]:
qsotemplate = volume+'nirspec-j1652-quasar-template.npy'
cube.makeqsotemplate(qsotemplate, radius=2.5, norm=1)

### 1.2. Setting up the fitting parameters <a class="anchor" id="chapter1_2"></a>

#### 1.2.1. Emission-line parameters <a class="anchor" id="chapter1_2_1"></a>

What lines do you want to fit? You can choose from the linelists available [here](https://github.com/Q3D/q3dfit/tree/main/data/linelists), or in `q3dfit/data/linelists/`.

In [ ]:
lines = ['HeII4686', 'Hbeta', '[OIII]4959', '[OIII]5007', 'Halpha','[NII]6583','[NII]6548','[SII]6716','[SII]6731','[OI]6300']

How many components do you want to be fitted to the emission lines?

In [ ]:
maxncomp = 3
argscheckcomp = {'sigcut': 5,'ignore': ['[OI]6300','[SII]6716', '[SII]6731']}

This block sets up initial conditions for the emission-line fit to each spaxel. Emission lines are set to a common redshift and velocity dispersion. However, different sets of emission lines can have different velocities and linewidths by specifying different lines to which to tie particular emission lines.

In [ ]:
linetie = dict()
ncomp = dict()
zinit_gas = dict()
siginit_gas = dict()
for i in lines:
    linetie[i] = '[OIII]5007'
    ncomp[i] = np.full((ncols,nrows),maxncomp)
    zinit_gas[i] = np.full((ncols,nrows,maxncomp),zsys_gas)
    siginit_gas[i] = np.full((ncols,nrows,maxncomp),50.)
    
# for 3 components    
for i in lines:
    zinit_gas[i][:,:,2] = 2.959
    siginit_gas[i][:,:,2] = 50.
    zinit_gas[i][:,:,1] = 2.9480
    siginit_gas[i][:,:,1] = 500.

# for 2 components
# for i in lines:
#     zinit_gas[i][:,:,1] = 2.959
#     siginit_gas[i][:,:,1] = 50.
#     zinit_gas[i][:,:,1] = 2.9480 #For 2 comp
#     siginit_gas[i][:,:,1] = 250. #for 2 comp


    
print(zinit_gas['Hbeta'][50,50])

`siglim_gas` sets lower and upper bounds for the Gaussian width (sigma) of the emission line. These limits can be set globablly, for all spaxels and components, by defining a 2-element array. The limits can also be set for individual spaxels (but all components) by defining an (Ncol x Nrow x 2) array.

In [ ]:
# Global limit
siglim_gas = np.array([40., 2000.])

# Spaxel-by-spaxel limit
# siglim_gas = np.ndarray((dx, dy, 2))
# siglim_gas[:,:,] = array([5.,1000.])
# siglim_gas[13, 10, :] = array([5.,500.])

#### Spectral resolution convolution
##### Just running convolution ---

If no convolution is desired, then set `spectres_convolve` to None. 
If convolution is desired, then `spectres_convolve` and `spect_instrum` are required variables.

Specify the desired convolution method using `spect_instrum`. The syntax is: 
`spect_instrum = {INSTRUMENT:[METHOD]}`, which should mirror the filename in `q3dfit/data/dispersion_files/`. METHOD specifies the grating, spectral resolution [R], Δλ FWHM in [Å], velocity in [km/s].

Examples convolving with: 
1. flat R=500: `spect_instrum = {'flat':['R500']}`
2. flat velocity FWHM = 30km/s: `spect_instrum = {'flat':['kms30']}`
3. flat Δλ FWHM = 4 Å: `spect_instrum = {'flat':['dlambda4']}`
4. JWST NIRSPEC / G140M: `spect_instrum = {'JWST_NIRSPEC':['G140M']}`

If desired flat convolution file does not exist, then a new file will be created automatically. 

In [ ]:
spect_instrum = {'JWST_NIRSPEC':['G235H']}
spectres_convolve = {'ws_instrum':spect_instrum}
# spectres_convolve = None

##### Creating convolution files (optional) ---

If the user wants to create a dispersion file, there are 2 methods:

1. First, run `from q3dfit.common.spectConvol import dispFile` and initialize: `dispObj = dispFile()`
2. Create the files with either method A or B

2.(A) Create a flat R=120 dispersion file from 10 μm to 30 μm: 

`dispObj.make_dispersion(120,WAVELEN=[10,30],TYPE='R')`

2.(B) Create a custom dispersion file with user supplied wavelength and Resolving power lists:

`dispObj.make_custom_dispersion([wavelen list],R=[some R list],FILENAME='custom_RlistA_disp')`

3. Specify desired dispersion file in convolution: running custom example 2B

`spect_instrum = {'custom':['RlistA']}`


In [ ]:
#from q3dfit.common.spectConvol import dispFile
#dobj = dispFile()
#dispObj.make_custom_dispersion([wavelen list],R=[some R list],FILENAME='custom_RlistA_disp')
#spect_instrum = {'custom':['RlistA']}

#### 1.2.2 Continuum parameters <a class="anchor" id="chapter1_2_2"></a>

`q3dfit` first masks emission lines before fitting. This sets is the default mask value in km/s for each velocity component for the first fit. During the second fit, the mask value is set automatically using the best-fit linewidths determined from the first fit.

In [ ]:
maskwidths_def = 2000.

The continuum fitting parameters specified here are for the case of simultaneous fitting of a stellar and quasar template.
- `hostord` = [Optional] additive Legendre polynomial for host [starlight] component. Adds components up to order `hostord`.
- `qsoord` = [Optional] additive Legendre polynomial for host [starlight] component. Adds components up to order `qsoord`.
- `qsoxdr` = Filename and path of quasar template created earlier with `makeqsotemplate.py`.

In [ ]:
fcncontfit = 'fitqsohost'
argscontfit = dict()
argscontfit['qsoxdr'] = qsotemplate
argscontfit['qsoord'] = 3
argscontfit['hostord'] = 3

### 1.3. Setting up the plotting parameters <a class="anchor" id="chapter1_3"></a>

One dictionary must be defined for a line plot to appear. `argspltlin1` holds the options for the first line plot, and `argspltlin2` is for the second. Parameter options are fed in as keyword/data pairs.

Required keywords in each dictionary:
* `nx`: Number of subplots in the horizontal direction
* `ny`: Number of subplots in the vertical direction
* Options for centerting the plot:
    - `line`: a string list of line labels
    - `center_obs`: a float list of wavelengths of each subplot center, in the observed (plotted) frame
    - `center_rest`: a float list of wavelengths of each subplot center, in the rest frame, which are converted to obs. frame

Optional keywords:
* `size`: float list of widths in wavelength space of each subplot; if not specified, default is 300 $Å$
* `IR`: set to `True` to use infrared-style plot

In [ ]:
argspltlin1 = {'nx': 3,
               'ny': 3,
               'line': ['HeII4686', 'Hbeta', '[OIII]5007','[OI]6300', 'Halpha','[SII]6716'],
               'size': [0.05, 0.05,0.07,0.07,0.07,0.07]
              }

The continuum plot can be changed by specifying several parameters. In this case, we have chosen to output a linear/linear plot of f$_\lambda$ vs. wavelength, in $\mu$m.

In [ ]:
argscontplot = {'xstyle': 'lin',
                'ystyle': 'lin',
                'fluxunit_out': 'flambda',
                'mode': 'dark'}

#### Line ratio constraints
`line1`, `line2`, and `comp` are required. `value` is the initial value of `line1`/`line2`, and `lower` and `upper` limits can also be specified. (If they are not, and the line pair is a doublet in the doublets.tbl file, then the lower and upper limits are set using the data in that file.) The ratio can also be `fixed`.

In [ ]:
# Required columns:
line1 = ['Halpha','[SII]6716']
line2 = ['Hbeta','[SII]6731']
comp = np.array([[0,1,2]]*2, dtype=np.int32)

value = [3.1,1.]
fixed = [False,False]
lower = [2.7,0.4]
upper = [100,1.5]



# Write table
from astropy.table import QTable
lineratio = QTable([line1, line2, comp, value, fixed,lower], names=['line1', 'line2', 'comp', 'value', 'fixed','lower'])

The following cell will create the initialisation structure. **Do not change anything in this cell.** 

In [ ]:
q3di = {# Required pars
        'fcninitpar': 'parinit',
        'fitran': fitrange,
        'fluxunits': 1,
        'infile': infile,
        'label': gal,
        'lines': lines,
        'linetie': linetie,
        'maxncomp': maxncomp,
        'name': gal,
        'ncomp': ncomp,
        'outdir': outdir,
        'zinit_gas': zinit_gas,
        'zsys_gas': zsys_gas,
        # Optional pars
        'datext':0,'varext':1,'dqext':2,
        'argscontfit': argscontfit,
        'argscontplot': argscontplot,
        'argsreadcube': argscube,
        'argspltlin1': argspltlin1,
        'cutrange': cutrange,
        'decompose_qso_fit': True,
        'fcncontfit': fcncontfit,
        'logfile': logfile,
        'maskwidths_def': maskwidths_def,
        'siglim_gas': siglim_gas,
        'siginit_gas': siginit_gas,
        'spect_convol':spectres_convolve,
         'argsinitpar': {'lineratio': lineratio},
        'argscheckcomp':argscheckcomp
        }

If you want to run `q3dfit` in batch mode, run this cell, which saves q3di to an `npy` file. In your python command line, read in file and run `q3dfit` with
<pre><code>q3di = '/path/to/the/npy/file/q3di.npy'
from q3dfit.q3df import q3df
q3df(q3di,cols=cols,rows=rows)</code></pre>

In [ ]:
q3di_npy = 'q3di.npy'
np.save(initdir+q3di_npy,q3di)

## 2. Run fitting and plotting routines <a class="anchor" id="chapter2"></a>

In [ ]:
from q3dfit.q3df import q3df
from q3dfit.q3da import q3da
import q3dfit.q3dpro as q3dpro

Select spaxels to fit based on a 2 sigma flux cut on peak [OIII] 5007 flux.

In [ ]:
from q3dfit.select_spaxels import mom8_map
import matplotlib.pyplot as plt
wavelength_segments = [[780,825]]
SNR_cut = 2
SNR_map,spaxels_to_fit = mom8_map(infile, datext=0, varext=1, dqext=None, wavext=None,
                   wmapext=None, plot=True, waveunit_in='micron',
                   waveunit_out='micron',wavelength_segments=wavelength_segments,SNR_cut=SNR_cut)
    

SNR_map[0:6] = 0
SNR_map[79:] = 0
SNR_map[:,72:] = 0

spaxels_to_fit = np.where(SNR_map>SNR_cut)

cols =list(spaxels_to_fit[0])
rows =list(spaxels_to_fit[1])
print(len(cols))
print(len(rows))

#plot the SNR map in log units
plt.imshow(np.log10(SNR_map.T),vmin=np.log10(2),vmax=np.log10(300),origin='lower')



Run the fit. Choose `quiet=False` for verbose output. An output dictionary for each spaxel is saved to a numpy binary file labeled with prefix `q3di['label']` and suffix `_col_row.npy`.

In [ ]:
#fitting spaxels using single core
#q3df(q3di,cols=cols,rows=rows, quiet=False)

#fitting spaxels using multicore
#q3df('nirspec-j1652-q3d_conv_fullcube3comp/q3di.npy',cols = cols, rows = rows, quiet=True,ncores=10)
#q3df(q3di,cols=cols,rows=rows, quiet=False)

#Fitting a single spaxel
q3df(q3di,cols=45,rows=29, quiet=False)

#Fitting all spaxels with no SNR cut
#q3df(q3di, quiet=False)

Plot the results. The resulting plots are saved as `.jpg` files. Data from multiple spaxels are collated into dictionaries containing emission-line and continuum best-fit parameters and saved to various ancillary files.

In [ ]:
#q3da on spaxels based on SNR cut
#q3da(q3di,cols = cols, rows = rows,noplots=True)

#q3da on individual spaxel
q3da(q3di,cols =45, rows = 29,noplots=False)

#q3da on all spaxels
#q3da(q3di, quiet=True, noplots=True)

In [ ]:
qpro = q3dpro.Q3Dpro(q3di, PLATESCALE=0.05, NOCONT=False)

do_kpc = False
saveFile = False
flx = [1e-10,1e-5]
qsocenter = None
pltarg = {'Ftot':flx,
          'Fci':flx,
          'Sig':[100,850],
          'v50':[-1000,1000],
          'w80':[100,850],
          'fluxlog':True,}

#qpro.make_linemap('Hbeta',XYSTYLE=do_kpc,xyCenter=qsocenter,
#                  LINEVAC=False,SAVEDATA=saveFile,VMINMAX=pltarg,PLTNUM=1,CMAP='inferno')
#qpro.make_linemap('[OIII]4959',XYSTYLE=do_kpc,xyCenter=qsocenter,
#                  LINEVAC=False,SAVEDATA=saveFile,VMINMAX=pltarg,PLTNUM=1,CMAP='inferno')
qpro.make_linemap('[OIII]5007',XYSTYLE=do_kpc,xyCenter=qsocenter,
                  LINEVAC=False,SAVEDATA=saveFile,VMINMAX=pltarg,PLTNUM=1,CMAP='inferno')
#qpro.make_linemap('Halpha',XYSTYLE=do_kpc,xyCenter=qsocenter,
#                  LINEVAC=False,SAVEDATA=saveFile,VMINMAX=pltarg,PLTNUM=1,CMAP='inferno')

In [ ]:
from matplotlib.pyplot import *
tight_layout()
figure(1,figsize=(12,12))
qpro.make_BPT(SNRCUT=10,PLTNUM=10,SAVEDATA=True)


In [ ]:
qpro.make_contmap()

In [ ]:
qpro.get_lineprop(LINESELECT='Halpha')

In [ ]:
from q3dfit.q3dpro import OneLineData
o3data = OneLineData(qpro.linedat, '[OIII]5007')
o3data.calc_cvdf(2.9489, [-1e3, 1e3], vstep=5)

In [ ]:
o3data.make_cvdf_map(50., velran=[-1e3, 1e3], markcenter=[40., 40.],
                     outfile=True)

In [ ]:
from matplotlib.pyplot import *
imshow(o3data.calc_cvdf_vel(90,calc_from_posvel=False).T-o3data.calc_cvdf_vel(10,calc_from_posvel=False).T,origin='lower',cmap='bwr',vmin=50,vmax=1500)
colorbar()